In [30]:
!pip install iapws

In [31]:
from iapws import IAPWS97

Steam enters a turbine steadily at 3 MPa and 450°C at a rate of 8 kg/s and exits at 0.2 MPa and 150°C. The steam is losing heat to the surrounding air at 100 kPa and 25°C at a rate of 300 kW, and the kinetic and potential energy changes are negligible. 

Determine: 
- (a) the actual power output,
- (b) the maximum possible power output,
- (c) the second-law efficiency,
- (d) the exergy destroyed, and
- (e) the exergy of the steam at the inlet conditions.

In [32]:
# inlet
p1_MPa = 3
T1_C = 450
T1_K = T1_C + 273.15

# outlet
p2_MPa = 0.2
T2_C = 150
T2_K = T2_C + 273.15

# environment
p0_MPa = 100/1000
T0_C = 25
T0_K = T0_C + 273.15

# heat losses
QdotOut_kW = 300

# mass flow rate
mdot_kgPerS = 8

We can determine the actual power output from our energy equation:

$$\frac{dE_\text{CV}}{dt} = \dot{E}_\text{in} - \dot{E}_\text{out}$$

$$\frac{dm_\text{CV}}{dt} = \dot{m}_\text{in} - \dot{m}_\text{out}$$

----------------------------------------------
Part (a)

----------------------------------------------

Assuming steady state:

$$ \dot{E}_\text{in} = \dot{E}_\text{out}$$

$$\dot{m}_\text{in} = \dot{m}_\text{out}$$

So we will refer to the mass flow rate as $\dot{m}$.

Our energy equation is then:

$$ (\dot{W} + \dot{Q})_\text{in} + \dot{m} (h_1 + w_1^2/2 + gz_1) = (\dot{W} + \dot{Q})_\text{out} + \dot{m} (h_2 + w_2^2/2 + gz_2)$$

We assume:
- Neglect kinetic and potential energy effects.
- No work transfer in.
- No heat transfer in.

$$\dot{W}_\text{out} = \dot{m} (h_1 - h_2) - \dot{Q}_\text{out}$$

In [33]:
water_1 = IAPWS97(T = T1_K, P = p1_MPa)
water_2 = IAPWS97(T = T2_K, P = p2_MPa)

In [34]:
WdotOut_kW = mdot_kgPerS * (water_1.h - water_2.h) - QdotOut_kW;
print('(a) WdotOut_kW = {0:1.1f} kW.'.format(WdotOut_kW))

(a) WdotOut_kW = 4304.6 kW.


----------------------------------------------
Part (b) 

----------------------------------------------
From the textbook we are told that the maximum power output is the reversible power. For this we need to analyze the system and its immediate surroundings, whose boundary is at the environmental temperature of $T_0$. The rate of change of exergy within the extended system is:

$$\frac{dX_\text{system}}{dt} = \dot{X}_\text{in} - \dot{X}_\text{out} - \dot{X}_\text{destroyed} $$

We assume the following:
- steady state
- reversible

Therefore, there is no exergy destroyed and no change of exergy in the system:

$$0 = \dot{X}_\text{in} - \dot{X}_\text{out} $$

The exergy rate for a fluid stream is (called flow exergy):

$$\dot{X} = \dot{W} + \left(1 - \frac{T_0}{T} \right)\dot{Q} + \dot{m} \psi$$

where 

$$\psi = (h - h_0) - T_0 (s - s_0) + w^2/2 + gz$$

If we neglect kinetic and potential energy effects, we can then write:

$$\dot{W}_\text{in} + \left(1 - \frac{T_0}{T_i} \right)\dot{Q}_\text{in} + \dot{m}\left[(h_1 - h_0) - T_0 (s_1 - s_0)\right] = \dot{W}_\text{out} + \left(1 - \frac{T_0}{T_j} \right)\dot{Q}_\text{out} + \dot{m}\left[(h_2 - h_0) - T_0 (s_2 - s_0)\right]$$

We can eliminate some terms (see assumptions from part(a)):

$$\dot{m}\left[(h_1 - h_0) - T_0 (s_1 - s_0)\right] = \dot{W}_\text{out} + \left(1 - \frac{T_0}{T_2} \right)\dot{Q}_\text{out} + \dot{m}\left[(h_2 - h_0) - T_0 (s_2 - s_0)\right]$$

Rearranging:

$$\dot{W}_\text{out} = \dot{m}\left[(h_1 - h_0) - T_0 (s_1 - s_0)\right] - \dot{m}\left[(h_2 - h_0) - T_0 (s_2 - s_0)\right] - \left(1 - \frac{T_0}{T_j} \right)\dot{Q}_\text{out}$$

$$\implies \dot{W}_\text{out} = \dot{m}\left[(h_1 - h_2) - T_0 (s_1 - s_2)\right] - \left(1 - \frac{T_0}{T_j} \right)\dot{Q}_\text{out}$$

So, which temperature do we select for $T_j$? We have to look at where the heat is being transferred to. In this case, we have heat losses from the turbine to the environment, which is at temperature $T_0$. Therefore, we set $T_j = T_0$. Substituting that into our equation above yields:
$$\dot{W}_\text{out} = \dot{m}\left[(h_1 - h_2) - T_0 (s_1 - s_2)\right] - \left(1 - \frac{T_j}{T_j} \right)\dot{Q}_\text{out},$$

and then,

$$\dot{W}_\text{out} = \dot{m}\left[(h_1 - h_2) - T_0 (s_1 - s_2)\right].$$

Another way of viewing the above—where the exergy heat term ends up being zero—is to recall the meaning of exergy. For exergy, we need to know the initial state, the process, and the dead state. If we want to maximize the work output given the initial and dead state, then we should have no losses. That is, we would expect to have no heat losses to the surrounding environment. Another way of thinking about it is what is the quality of heat that is lost to the environment? It is useless, therefore, we should assign it zero value. In this case, heat lost to the environment has zero exergy.

Substituting yields:

In [35]:
WdotRev_kW = mdot_kgPerS*((water_1.h - water_2.h)-T0_K*(water_1.s - water_2.s))
print('(b) WdotRev_kW = {0:1.1f} kW.'.format(WdotRev_kW))

(b) WdotRev_kW = 5071.1 kW.


--------------
part (c) 

--------------

Since we know the real power output (part (a))and the reversible power output (part (b)), we can determine the second law efficiency of this turbine:

$$\eta_\text{II} = \frac{\dot{W}_\text{out}}{\dot{W}_\text{rev,out}} $$

In [36]:
eta_II = WdotOut_kW/WdotRev_kW
print('(c) eta_II = {0:1.1f} %.'.format(eta_II*100))

(c) eta_II = 84.9 %.


This means that approximately 15% of the work potential is wasted during this process.

-------
part (d)

-------

Recall the definition of exergy destruction:

$$\frac{dX_\text{system}}{dt} = \dot{X}_\text{in} - \dot{X}_\text{out} - \dot{X}_\text{destroyed} $$

$$\implies \dot{X}_\text{destroyed} = \dot{X}_\text{in} - \dot{X}_\text{out}$$

So, the exergy transferred in is the answer we have from part (b). Essentially, the exergy transferred in is the maximum useful work that can be transferred out of the system. The exergy transferred out is the actual useful work transferred out of the system. Therefore, 

$$\dot{X}_\text{destroyed} = \dot{W}_\text{rev,out} - \dot{W}_\text{out}$$


In [37]:
Xdestroyed_kW = WdotRev_kW - WdotOut_kW
print('(d) Xdestroyed_kW = {0:1.1f} kW.'.format(Xdestroyed_kW))

(d) Xdestroyed_kW = 766.5 kW.


Note: The answer given in the book is incorrect (776 kW). 

-------
part (d), alternative solution

-------

We can also determine the rate of exergy destruction by determining the rate of entropy generation. Recall:

$\frac{dS_\text{CV}}{dt} = \dot{S}_\text{in} - \dot{S}_\text{out} + \dot{S}_\text{gen} $

Assume: steady state

$\implies \dot{S}_\text{gen} = \dot{S}_\text{out} - \dot{S}_\text{in} = \dot{m} \left(s_2 - s_1 \right) + \frac{\dot{Q}_\text{out}}{T_0}$

Why this formula? The $\dot{m}(s_2 - s_1)$ term is the change of entropy in the control volume associated with the flow stream. We also have heat being transferred out of the turbine to the environment which is at temperature $T_0$ and is the term on the right hand side.

In [38]:
Sgen_kWperK = mdot_kgPerS*(water_2.s - water_1.s) + QdotOut_kW/T0_K
print(Sgen_kWperK)

2.570931104794056


Recall the relation between entropy generation and exergy destruction:
$\dot{X}_\text{destroyed} = T_0 \dot{S}_\text{gen}$


In [39]:
Xdestroyed_kW_ = Sgen_kWperK*T0_K
print('(d) Alternative solution: Xdestroyed_kW_ = {0:1.1f} kW.'.format(Xdestroyed_kW_))

(d) Alternative solution: Xdestroyed_kW_ = 766.5 kW.
